# Installing and importing modules

In [ ]:
!pip install elasticsearch
!pip install jproperties

In [71]:
from elasticsearch import Elasticsearch
from jproperties import Properties
import pickle
from elasticsearch import helpers

from tqdm import tqdm

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading respective pickle files

In [25]:
config = Properties()
with open('/content/drive/MyDrive/es_credentials.prop','rb') as f:
  config.load(f)

In [33]:
with open('/content/drive/MyDrive/Pickle files/Product_level_df','rb') as f:
  prod_df = pickle.load(f)

In [35]:
with open('/content/drive/MyDrive/Pickle files/final_data_frame','rb') as f:
  image_df = pickle.load(f)

# Elastic Search

In [77]:
es = Elasticsearch([{'host':'ec2-18-217-197-173.us-east-2.compute.amazonaws.com','port':'9200'}])

In [80]:
es.ping()

True

In [37]:
prod_df_json = prod_df.to_dict('records')

In [38]:
image_df_json = image_df.to_dict('records')

In [59]:
es.indices.create(index='product-level',ignore=400)

{'acknowledged': True, 'index': 'product-level', 'shards_acknowledged': True}

In [74]:
es.indices.create(index='image-level',ignore=400)

{'acknowledged': True, 'index': 'image-level', 'shards_acknowledged': True}

In [79]:
prod_df_json[0]

{'category': 'dresses',
 'gender': 'women',
 'path': 'women/dresses/casual_and_day_dresses/56037632/56037632_0.jpeg',
 'product-name': 'black knit midi dress',
 'product-number': '56037632',
 'sub-category': 'casual_and_day_dresses'}

# uploading data into ES

In [81]:
def generator_prod(df):
  for i,x in enumerate(df):
    yield{
        '_index':'product-level',
        '_type':'_doc',
        '_id': i+1,
        '_source':{
            "product-number":x.get("sub-category",""),
            "product-name":x.get("product-name",""),
            "category":x.get("category",""),
            "sub-category":x.get("sub-category",""),
            "image-path":x.get("path","")
        }
    }
  raise StopIteration

In [82]:
try:
  res1 = helpers.bulk(es,generator_prod(prod_df_json))
  print('working')
except Exception as e:
  pass

/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [83]:
def generator_image(df):
  for i,x in enumerate(df):
    yield{
        '_index':'image-level',
        '_type':'_doc',
        '_id': i+1,
        '_source':{
            "product-number":x.get("sub-category",""),
            "product-name":x.get("product-name",""),
            "category":x.get("category",""),
            "sub-category":x.get("sub-category",""),
            "image-path":x.get("path","")
        }
    }
  raise StopIteration

In [84]:
try:
  res2 = helpers.bulk(es,generator_image(image_df_json))
  print('working')
except Exception as e:
  pass

/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)
